## In this notebook:

* 3 input dataframe: chr18, 19, 20
* using all numeric scores for predictors
* fully connected neural net
* training with dataloading and epochs
* aucuracy assessement of .954 on held-out test 

### Imports

In [1]:
#!pip install numpy 
#!pip install pandas
#!pip install torch
#!pip install sklearn

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.optim as optim

### Manipulating data
* Import csv we want
* Map our features to usuable integers
* Map our labels to a one hot encoding

In [2]:
#################################################### 
df = pd.read_pickle("./pickled/chr2-4_featureset1.pkl")
df = df.replace({'-': 0})

# Split our data into two separate sets for training purposes
train, test = train_test_split(df, shuffle=False)

print(test.head())
print(train.shape, test.shape)

                      aapos TSL codonpos HUVEC_fitCons_score  \
chr pos      ref alt                                           
3   37020309 G   A       -1   2        0            0.117559   
                 C       -1   2        0            0.117559   
    37020312 T   G       55   2        2            0.714379   
    37020314 G   T       56   2        1            0.714379   
    37020326 C   T       60   2        1            0.714379   

                         ExAC_cnv.score         GDI LoFtool_score  \
chr pos      ref alt                                                
3   37020309 G   A    0.255578617187788  1715.16552      1.65E-03   
                 C    0.255578617187788  1715.16552      1.65E-03   
    37020312 T   G    0.255578617187788  1715.16552      1.65E-03   
    37020314 G   T    0.255578617187788  1715.16552      1.65E-03   
    37020326 C   T    0.255578617187788  1715.16552      1.65E-03   

                      SORVA_LOF_MAF0.005_HetOrHom  Problematic  
ch

### DataLoader
* Implement the Dataset class supplied by PyTorch
  * Used Map style
  * API: https://pytorch.org/docs/stable/data.html#map-style-datasets
* Instanciate DataLoader using Dataset instance

In [5]:
class DataSetdbNSFP(Dataset):
    def __init__(self, data):
        self.data = torch.FloatTensor(data.values.astype('float'))
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
            target = self.data[index][-1]
            data_val = self.data[index] [:-1]
            return data_val,target
        
train_dataset = DataSetdbNSFP(train)
valid_dataset = DataSetdbNSFP(train)
test_dataset = DataSetdbNSFP(test)

train_loader = DataLoader(train_dataset, batch_size=59, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=59, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=59, shuffle=True)


### Model
* This model is a simple implementation
* Based on https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [9]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # In: nb of features, out: nb of examples
        self.fc = nn.Linear(8, 1000)
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 1000)
        self.fc4 = nn.Linear(1000, 1000)
         # In: nb of examples, out: nb of predictions
        self.fc5 = nn.Linear(1000, 2)

    def forward(self, x):
        x = x.view((-1, 8))
        h = F.relu(self.fc(x))
        h = self.fc2(h)
        h = F.relu(self.fc2(h))
        h = self.fc3(h)
        h = F.relu(self.fc3(h))
        h = self.fc4(h)
        h = F.relu(self.fc4(h))
        h = self.fc5(h)
        # Softmax to get the actual labels
        return F.softmax(h, dim=1)  ## Note: can potentially use this to get confidence of category predicted 
    
model = Model()
# if cuda:
#     model.cuda()

# Optimizer based on Adam algorithm, uses a slightly lower rate
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Training
* Run a fixed number of epochs
* After each epoch, run the validation set to adjust learning
* Print results
* Based on example: https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [10]:
EPOCHS = 20

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        print(data.shape)
        print(target.shape)
        
#         if cuda:
#             data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        y_pred = model(data) 
        
        # Calculate and appy loss
        loss = F.cross_entropy(y_pred, target.long())
        loss.backward()
        optimizer.step()

        print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1,
            EPOCHS,
            batch_idx * len(data), 
            len(train_loader.dataset),
            100. * batch_idx / len(train_loader), 
            loss.cpu().data.item()))
    
    # NOTE: The Variable class is a wrapper class around a tensor with the added functionalities of back propagation
    # For x, take all cols of datafram except the last one
    evaluate_x = Variable(valid_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
    print(evaluate_x)
    # For y, take last col
    evaluate_y = Variable(valid_loader.dataset.data[:, -1:])
#     if cuda:
#         evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()
    model.eval()
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    
    y_labels = torch.flatten(evaluate_y.data)
    print('Predictions:', pred)
    print('Actual values:', y_labels)
    d = pred.eq(y_labels).cpu()
    accuracy = d.sum().item()*1./d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Validation Accuracy: {:.2f}%'.format(
        epoch+1,
        EPOCHS,
        len(train_loader.dataset), 
        len(train_loader.dataset),
        100. * batch_idx / len(train_loader), 
        loss.cpu().data.item(),
        accuracy*100))

torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [0/8793 (0%)]	Loss: 0.980809
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [59/8793 (1%)]	Loss: 0.656881
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [118/8793 (1%)]	Loss: 0.500114
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [177/8793 (2%)]	Loss: 0.532628
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [236/8793 (3%)]	Loss: 0.554346
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [295/8793 (3%)]	Loss: 0.636637
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [354/8793 (4%)]	Loss: 0.483398
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [413/8793 (5%)]	Loss: 0.635300
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [472/8793 (5%)]	Loss: 0.568264
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [531/8793 (6%)]	Loss: 0.669249
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [590/8793 (7%)]	Loss: 0.664262
torch.Size([59, 8])
torch.Size([59])
 Train Ep

 Train Epoch: 1/20 [5487/8793 (62%)]	Loss: 0.584448
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [5546/8793 (63%)]	Loss: 0.453281
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [5605/8793 (63%)]	Loss: 0.618345
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [5664/8793 (64%)]	Loss: 0.550689
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [5723/8793 (65%)]	Loss: 0.584458
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [5782/8793 (65%)]	Loss: 0.533727
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [5841/8793 (66%)]	Loss: 0.601397
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [5900/8793 (67%)]	Loss: 0.567499
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [5959/8793 (67%)]	Loss: 0.720048
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [6018/8793 (68%)]	Loss: 0.550550
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [6077/8793 (69%)]	Loss: 0.584448
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 1/20 [6

 Train Epoch: 2/20 [1593/8793 (18%)]	Loss: 0.601425
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [1652/8793 (19%)]	Loss: 0.516651
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [1711/8793 (19%)]	Loss: 0.618356
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [1770/8793 (20%)]	Loss: 0.550550
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [1829/8793 (21%)]	Loss: 0.654359
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [1888/8793 (21%)]	Loss: 0.567499
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [1947/8793 (22%)]	Loss: 0.703092
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [2006/8793 (23%)]	Loss: 0.482753
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [2065/8793 (23%)]	Loss: 0.516651
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [2124/8793 (24%)]	Loss: 0.569272
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [2183/8793 (25%)]	Loss: 0.720042
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [2

 Train Epoch: 2/20 [7080/8793 (80%)]	Loss: 0.533600
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [7139/8793 (81%)]	Loss: 0.499702
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [7198/8793 (81%)]	Loss: 0.533600
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [7257/8793 (82%)]	Loss: 0.567499
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [7316/8793 (83%)]	Loss: 0.550550
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [7375/8793 (83%)]	Loss: 0.584448
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [7434/8793 (84%)]	Loss: 0.533600
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [7493/8793 (85%)]	Loss: 0.635295
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [7552/8793 (85%)]	Loss: 0.669194
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [7611/8793 (86%)]	Loss: 0.652245
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [7670/8793 (87%)]	Loss: 0.550550
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 2/20 [7

 Train Epoch: 3/20 [3186/8793 (36%)]	Loss: 0.618346
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 3/20 [3245/8793 (37%)]	Loss: 0.601397
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 3/20 [3304/8793 (37%)]	Loss: 0.601397
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 3/20 [3363/8793 (38%)]	Loss: 0.567499
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 3/20 [3422/8793 (39%)]	Loss: 0.584448
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 3/20 [3481/8793 (39%)]	Loss: 0.516651
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 3/20 [3540/8793 (40%)]	Loss: 0.601397
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 3/20 [3599/8793 (41%)]	Loss: 0.736991
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 3/20 [3658/8793 (41%)]	Loss: 0.618346
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 3/20 [3717/8793 (42%)]	Loss: 0.669194
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 3/20 [3776/8793 (43%)]	Loss: 0.601397
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 3/20 [3

...

 Train Epoch: 18/20 [4248/8793 (48%)]	Loss: 0.635295
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 18/20 [4307/8793 (49%)]	Loss: 0.601397
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 18/20 [4366/8793 (49%)]	Loss: 0.601397
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 18/20 [4425/8793 (50%)]	Loss: 0.567499
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 18/20 [4484/8793 (51%)]	Loss: 0.499702
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 18/20 [4543/8793 (51%)]	Loss: 0.652245
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 18/20 [4602/8793 (52%)]	Loss: 0.669194
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 18/20 [4661/8793 (53%)]	Loss: 0.618346
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 18/20 [4720/8793 (53%)]	Loss: 0.601397
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 18/20 [4779/8793 (54%)]	Loss: 0.550550
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 18/20 [4838/8793 (55%)]	Loss: 0.618346
torch.Size([59, 8])
torch.Size([59])
 Train Epo

 Train Epoch: 19/20 [295/8793 (3%)]	Loss: 0.669194
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [354/8793 (4%)]	Loss: 0.652245
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [413/8793 (5%)]	Loss: 0.499702
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [472/8793 (5%)]	Loss: 0.567499
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [531/8793 (6%)]	Loss: 0.635296
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [590/8793 (7%)]	Loss: 0.635296
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [649/8793 (7%)]	Loss: 0.567499
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [708/8793 (8%)]	Loss: 0.499702
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [767/8793 (9%)]	Loss: 0.499702
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [826/8793 (9%)]	Loss: 0.652245
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [885/8793 (10%)]	Loss: 0.618346
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [944/8793 (

 Train Epoch: 19/20 [5723/8793 (65%)]	Loss: 0.550550
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [5782/8793 (65%)]	Loss: 0.567499
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [5841/8793 (66%)]	Loss: 0.482753
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [5900/8793 (67%)]	Loss: 0.567499
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [5959/8793 (67%)]	Loss: 0.499702
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [6018/8793 (68%)]	Loss: 0.686143
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [6077/8793 (69%)]	Loss: 0.635296
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [6136/8793 (69%)]	Loss: 0.567499
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [6195/8793 (70%)]	Loss: 0.703092
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [6254/8793 (71%)]	Loss: 0.584448
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 19/20 [6313/8793 (71%)]	Loss: 0.482753
torch.Size([59, 8])
torch.Size([59])
 Train Epo

 Train Epoch: 20/20 [1770/8793 (20%)]	Loss: 0.584448
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [1829/8793 (21%)]	Loss: 0.550550
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [1888/8793 (21%)]	Loss: 0.584448
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [1947/8793 (22%)]	Loss: 0.601397
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [2006/8793 (23%)]	Loss: 0.584448
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [2065/8793 (23%)]	Loss: 0.669194
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [2124/8793 (24%)]	Loss: 0.618346
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [2183/8793 (25%)]	Loss: 0.601397
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [2242/8793 (25%)]	Loss: 0.686143
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [2301/8793 (26%)]	Loss: 0.550550
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [2360/8793 (27%)]	Loss: 0.584448
torch.Size([59, 8])
torch.Size([59])
 Train Epo

 Train Epoch: 20/20 [7198/8793 (81%)]	Loss: 0.584448
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [7257/8793 (82%)]	Loss: 0.635296
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [7316/8793 (83%)]	Loss: 0.482753
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [7375/8793 (83%)]	Loss: 0.618346
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [7434/8793 (84%)]	Loss: 0.567499
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [7493/8793 (85%)]	Loss: 0.550550
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [7552/8793 (85%)]	Loss: 0.567499
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [7611/8793 (86%)]	Loss: 0.550550
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [7670/8793 (87%)]	Loss: 0.635296
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [7729/8793 (87%)]	Loss: 0.584448
torch.Size([59, 8])
torch.Size([59])
 Train Epoch: 20/20 [7788/8793 (88%)]	Loss: 0.533600
torch.Size([59, 8])
torch.Size([59])
 Train Epo

### Testing
* Take testing dataset, run it against current model
* Evaluate testing accuracy

In [11]:
# Follow same steps as when we test with validation set, except with the test set
evaluate_x = Variable(test_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
evaluate_y = Variable(test_loader.dataset.data[:, -1:])

model.eval()
output = model(evaluate_x)
pred = output.data.max(1)[1]

y_labels = torch.flatten(evaluate_y.data)
d = pred.eq(y_labels).cpu()
accuracy = d.sum().item()*1./d.size()[0]  ## Accuracy = (TP+TN)/(TP+TN+FP+FN)

print('\n Testing Accuracy: {:.2f}%'.format(

    accuracy*100))


 Testing Accuracy: 73.08%


### Metrics
* Use sklearn builtins to calculate different metrics

In [12]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
print("Confusion matrix: \n", confusion_matrix(y_labels, pred))

# Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_labels, pred))

# Recall
from sklearn.metrics import recall_score
recall = recall_score(y_labels, pred, average=None)
print("Recall TP/(TP+FN):", recall)

# Precision
from sklearn.metrics import precision_score
precision = precision_score(y_labels, pred, average=None)
print("Precision TP/(TP+FP):",precision)


# F1 score
F1 = 2 * (precision * recall) / (precision + recall)
print("F1:", F1)

Confusion matrix: 
 [[   0  789]
 [   0 2142]]
Accuracy: 0.7308085977482088
Recall TP/(TP+FN): [0. 1.]
Precision TP/(TP+FP): [0.        0.7308086]
F1: [       nan 0.84447073]


c:\users\vincent\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\vincent\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
